In [35]:
import pandas as pd

In [34]:
df = pd.read_csv("twitter_training.csv")

In [36]:
df

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [31]:
df.drop(['2401', 'Borderlands'], axis=1, inplace=True)
df = df.sample(1000)
df.reset_index(drop=True, inplace=True)

In [32]:
df['im getting on borderlands and i will murder you all ,']

0      @Beluba @NBA2K @Ronnie2K LR @2Kstauff @NBA2K_2...
1                That Assassin’s Creed flag is so cool 🥺
2      I'm just seeing what the Overwatch community i...
3      Our PUBG banned tourism in Pakistan. Pubg love...
4      PS5 alternatives to Amazon are a fucking joke....
                             ...                        
995    @FortniteGame You actually ruined the game for...
996    A Missouri appeals court on Tuesday ordered Jo...
997                                               am smh
998    49 Somehow our Fifa 20 is becoming a bigger sh...
999    Also bought Borderlands on the switch!! love t...
Name: im getting on borderlands and i will murder you all ,, Length: 1000, dtype: object

In [33]:
df = df.rename(columns={'im getting on borderlands and i will murder you all ,': 'sentence'})
df = df.rename(columns={'Positive': 'sentiment'})

In [7]:
import string
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tag_dict = {'J': wordnet.ADJ,'N': wordnet.NOUN,'V': wordnet.VERB,'R': wordnet.ADV}

In [8]:
def preprocess_text(text):
    text = str(text).lower()
    clean_text = re.sub('https?://\S+|www\.\S+', '', text) #URLs
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), '', clean_text) #Punctuations
    clean_text = re.sub('\n', '', clean_text) #Backslash n
    clean_text = [word for word in clean_text.split(' ') if word not in stop_words] #Remove stopwords
    clean_text=" ".join(clean_text)
    tokens = clean_text.split()
    pos_tags = nltk.pos_tag(tokens)
    wordnet_tags = [(token, tag_dict.get(tag[0].upper(), wordnet.NOUN)) for token, tag in pos_tags]
    tokens = [lemmatizer.lemmatize(token, tag) for token, tag in wordnet_tags] #Lemmatize
    return " ".join(tokens)

In [9]:
df['clean_sentence'] = df['sentence'].apply(preprocess_text)

In [10]:
df['enc_sentiment'] = df['sentiment'].map({'Positive': 1, 'Negative': 0, 'Neutral': 2, 'Irrelevant': 3})

In [11]:
df['enc_sentiment'].unique()

array([2, 1, 0, 3], dtype=int64)

In [ ]:
df

In [12]:
docs = df['clean_sentence']
labels = df['enc_sentiment']
from sklearn.model_selection import train_test_split
x_train, x_test , y_train, y_test = train_test_split(docs, labels , test_size = 0.20)

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 13938
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x_train)

In [26]:
x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)

In [ ]:
x_train_padded = pad_sequences(x_train_sequences,maxlen=1000, padding="post",
                       truncating="post")
x_test_padded = pad_sequences(x_test_sequences,maxlen=1000,
                               padding="post", truncating="post")

In [ ]:
x_train_padded.shape, x_test_padded.shape

((800, 1000), (200, 1000))

In [ ]:
embed_size = 300

In [69]:
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, embed_size , input_length=1000))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 1000, 300)         1023000   
                                                                 
 conv1d_10 (Conv1D)          (None, 1000, 64)          76864     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 500, 64)          0         
 1D)                                                             
                                                                 
 conv1d_11 (Conv1D)          (None, 500, 32)           8224      
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 250, 32)          0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 8000)             

In [71]:
history = model.fit(x_train_padded, y_train, epochs=10)

Epoch 1/10


25/25 [==============================] - 2s 20ms/step - loss: -146.6523 - accuracy: 0.2812
Epoch 2/10
25/25 [==============================] - 1s 25ms/step - loss: -9656.0049 - accuracy: 0.2763
Epoch 3/10
25/25 [==============================] - 1s 23ms/step - loss: -156092.4375 - accuracy: 0.2763
Epoch 4/10
25/25 [==============================] - 1s 21ms/step - loss: -1117829.8750 - accuracy: 0.2763
Epoch 5/10
25/25 [==============================] - 1s 20ms/step - loss: -5543438.0000 - accuracy: 0.2763
Epoch 6/10
25/25 [==============================] - 0s 19ms/step - loss: -19393646.0000 - accuracy: 0.2763
Epoch 7/10
25/25 [==============================] - 0s 18ms/step - loss: -53658972.0000 - accuracy: 0.2763
Epoch 8/10
25/25 [==============================] - 0s 18ms/step - loss: -133800032.0000 - accuracy: 0.2763
Epoch 9/10
25/25 [==============================] - 0s 16ms/step - loss: -315008256.0000 - accuracy: 0.2763
Epoch 10/10
25/25 [==============================] - 0s 16m

In [ ]:
x_train

In [ ]:
y_train